# Analiza - sprawdzenie bardzo prostej strategii opartej jedynie na dwóch formacjach świecowych + "zasada 11/20" + dodatkowo RSI

<h3>
<center>
Robert Nowotniak <rnowotniak@gmail.com><br/>
w oparciu o metodykę Fraktal Trader Moduł 1, formacje świecowe
</center>
</h3>

* [Eksperyment 1) Analiza młotków - EURUSD 1h](#eksperyment1)

* [Eksperyment 2) Spadające gwiazdy - Dow Jones 1h](#eksperyment2)

* [Eksperyment 3) Analiza zbiorcza dla wszystkich walorów - Młotki](#eksperyment3)

* [Eksperyment 3) - Podsumowanie zbiorcze](#eksperyment3summary)
 
* [Eksperyment 4) Analiza zbiorcza dla wszystkich walorów - Spadające gwiazdy](#eksperyment4)

* [Eksperyment 4) - Podsumowanie zbiorcze](#eksperyment4summary)

In [5]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import talib
import talib.abstract as ta
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import seaborn as sns
import glob, os.path
sns.set_style('whitegrid')
%matplotlib inline

In [6]:
def ohlc(r):
    return r['open'], r['high'], r['low'], r['close']

## Wczytanie specyfikacji wszystkich instrumentów

Specyfikacja instrumentów wygenerowana w MetaTraderze skryptem MQL allmarketdata.ex4

In [7]:
specs = pd.read_csv('instruments.csv', sep=';', index_col=0)

# XXX - nie da się tego wyciągnąć bezpośrednio ze specyfikacji ??
def setPipSize(x):
    if x['Pr. Mode'] == 'Forex':
        return 0.0001
    return float(x['Tick Size']) # XXX - uproszczenie, nie jestem pewien, czy to jest prawidłowe
    if x['Symbol'].endswith('JPY'):
            return 0.01
        # elif x['Symbol'].endswith('HUF'):  # XXX Hungarian Forint ???
        #return 0.0001
    return 1

specs['Pip'] = specs.apply(setPipSize, axis=1)

### Wczytanie historycznych danych wszystkich walorów do klasy Instrument

In [8]:
class Instrument:
    def __init__(self, name):
        self.name = name
        self.spec = None
        self.results = None
        self.balance = 0  # +/- gain value
        self.stats = {} # PROFIT: ...,  LOSS: ...
        self.success_rate = None
        self.data = pd.read_csv('data/%s60.csv' % name, index_col = False,
                   names=['day', 'hour', 'open', 'high', 'low', 'close', 'volume'],
                               dtype={'open':np.float64,'high':np.float64,
                                     'low':np.float64,'close':np.float64,'volume':np.float64}
                               )
        self.data['date'] = pd.DatetimeIndex(self.data.day + ' ' + self.data.hour)
        del self.data['day']
        del self.data['hour']
        self.data.set_index('date', inplace = True)
        
        # Calculate RSI14 for this instrument
        self.rsi = ta.RSI(self.data, 14)
    
    def __str__(self):
        s = self.name
        if self.data is not None:
            s = s + ' ' + str(self.data.shape)
        return s

INSTRUMENTS = {}        
        
for f in glob.glob('data/*60.csv'):
    name = os.path.basename(f).replace('60.csv', '')
    print(name)
    inst = Instrument(name)
    inst.spec = specs.ix[name]
    INSTRUMENTS[name] = inst
 

USDCHF
EURDKK
EURUSD
EURPLN
EURNOK
COCOA-SEP17
GER30Cash
EURCHF
BRENT-AUG17
POL20Cash
GBPCAD
SUGAR-JUL17
CADJPY
GOLD
EURRUB
SILVER
AUDCAD
US30Cash
EURSEK
AUDJPY
AUDCHF
EURAUD
USDCAD
EURHKD
AUDNZD


In [40]:
INSTRUMENTS['EURRUB'].rsi['2017/06/16 10:00']

67.19948864159953

In [9]:
INSTRUMENTS['EURNOK'].spec

Description      Euro vs Norwegian Krone
Digits                                 5
Spread                               614
Swap Long                         -85.69
Swap Short                        -30.59
Swap Type                         points
Stop Level                           620
Lot Size                          100000
Lot Min                             0.01
Lot Step                            0.01
Tick Value                      0.443566
Tick Size                          1e-05
Trade Allowed                        yes
Pr. Mode                           Forex
Pip                               0.0001
Name: EURNOK, dtype: object

Delete all non-currencies:

In [27]:
todelete = []
for iname in INSTRUMENTS:
    if INSTRUMENTS[iname].spec['Pr. Mode'] != 'Forex':
        todelete.append(iname)
for iname in todelete:
    del INSTRUMENTS[iname]
for iname in INSTRUMENTS:
    print(iname)

EURSEK
AUDJPY
USDCHF
EURUSD
GBPCAD
EURAUD
AUDCAD
EURPLN
USDCAD
EURNOK
EURCHF
AUDCHF
AUDNZD
CADJPY


## Zasada 11 / 20 - porównywanie rozmiaru świecy z 20 poprzednimi

In [14]:
def zasada1120(inst, t):
    
    BACKSIZE = 20         # XXX Wyciągnąć gdzieś do konfiguracji
    CANDLES_REQUIRED = 11 # XXX
    
    t = pd.Timestamp(t)
    i = inst.data.index.get_loc(t, 'nearest')
    o, h, l, c = ohlc(inst.data.iloc[i])
    size = h - l
   
    j = 1
    smaller_candles = 0
    while j <= BACKSIZE and i - j >= 0 and smaller_candles < CANDLES_REQUIRED:
        o, h, l, c = ohlc(inst.data.iloc[i - j])
        if size > h - l:
            smaller_candles += 1
        j += 1
    if smaller_candles >= CANDLES_REQUIRED:
        return True
    
    return False
    
zasada1120(INSTRUMENTS['US30Cash'], '2017/06/20 22:00')

True

## Sprawdzanie, czy dany sygnał był prawidłowy(czy zarobił) -- B. WAŻNY FRAGMENT
### (ta funkcja docelowo byłaby jednym z głównych elementów pełnego symulatora strategii)

In [47]:
# returns:
#   (PROFIT/LOSS, gain value, nswaps, sp, tp, size)
def sprawdz(inst, t, buysell = 'buy'):
    t = pd.Timestamp(t)
    i = inst.data.index.get_loc(t, 'nearest')
    o, h, l, c = ohlc(inst.data.iloc[i])
    c0 = c
    size = h - l
    
    spread = 3 # XXX  US30
    spread = 0.0003 # XXX EURUSD
    spread = float(inst.spec['Spread']) / 10**int(inst.spec['Digits'])  # XXX
    #print(spread)
    PROFIT_TO_RISK = 3  # XXX Wyciagnac do konfiguracji
    
    if buysell == 'buy':
        sl = l - inst.spec['Pip'] # 1 punkt pod formacja
        risk = c - sl + spread # -> mozliwa strata = roznica poziomow + spread
        tp = c + spread + risk * PROFIT_TO_RISK # -> mozliwy zysk = roznica poziomow - spread = risk*PTR
    elif buysell == 'sell':
        sl = h + inst.spec['Pip'] + spread # 1 punkt nad formacja + spread (chyba z zalozenia ze spada szybciej)
        risk = sl - c + spread # mozliwa strata = roznica poziomow + spread
        tp = c - risk * PROFIT_TO_RISK - spread  # -> mozliwy zysk = roznica poziomow - spread = risk*PTR
    else:
        raise Exception('buysell')
    # print(sl, tp)
    
    before_midnght = inst.data.index[i].hour > 12
    nswaps = 0
    while i < inst.data.shape[0] - 1:
        i += 1
        o, h, l, c = ohlc(inst.data.iloc[i])
        
        if before_midnght and inst.data.index[i].hour < 12:
            # print('SWAP')
            nswaps += 1
        before_midnght = inst.data.index[i].hour > 12
        
        result = None
        if (buysell == 'buy' and h >= tp) or (buysell == 'sell' and l <= tp):
            # print(inst.data.index[i])
            result = 'PROFIT'
            
        # Sprawdzenie warunku 10pips
        how_many_pips = 10.
        if h - c0 >= how_many_pips * inst.spec['Pip'] and buysell == 'buy':
            result = 'PROFIT'
            return result, how_many_pips * inst.spec['Pip'] - spread, nswaps, sl, tp, size 
        if c0 - l >= 10. * inst.spec['Pip'] and buysell == 'sell':
            result = 'PROFIT'
            return result, how_many_pips * inst.spec['Pip'] - spread, nswaps, sl, tp, size     
        #################################
            
        if (buysell == 'buy' and l <= sl) or (buysell == 'sell' and h >= sl):
            # print(inst.data.index[i])
            if result == 'PROFIT':
                return 'Unknown'
            result = 'LOSS'
        if result == 'PROFIT':
            return result, PROFIT_TO_RISK * risk, nswaps, sl, tp, size #, zasada1120(inst, t)
        if result == 'LOSS':
            return result, -risk, nswaps, sl, tp, size #, zasada1120(inst, t)

t = '2017/06/22 01:00'
print(sprawdz(INSTRUMENTS['EURUSD'], t))

('LOSS', -0.00042000000000001802, 0, 1.11635, 1.1180300000000001, 0.00029000000000012349)


## Definicja wybranych formacji świecowych - funkcje wykrywające

In [30]:
def jestMlotek(row):
    o, h, l, c = ohlc(row)
    if c > o and c - o <= 0.4 * (h - l):
        if h - c <= 0.14 * (h - l):
            return True
    if c < o and o - c <= 0.4 * (h - l):
        if h - o <= 0.14 * (h - l):
            return True    
    return False

def jestSpadajacaGwiazda(row):
    o, h, l, c = ohlc(row)
    if c > o and c - o <= 0.4 * (h - l):
        if o - l <= 0.14 * (h - l):
            return True
    if c < o and o - c <= 0.4 * (h - l):
        if c - l <= 0.14 * (h - l):
            return True    
    return False

# ...
# Tutaj dalej docelowo powinny być zaimplementowane inne formacje (przenikania, zasłona ciemnej chmury itp)
# ...

<a id="eksperyment1"></a>
## Eksperyment 1) Analiza młotków - EURUSD 1h

In [48]:
inst = INSTRUMENTS['EURUSD']

stats = {}
balance = 0.
for idx, row in inst.data['2017/01/01':'2017/07/01'].iterrows():
    if jestMlotek(row) and zasada1120(inst,idx) and inst.rsi[idx] <= 30:
        res = sprawdz(inst, idx)
        if not res:
            print(idx, ' ', res)
            continue        
        print(idx, ' ', '%6s'%res[0], ' ', end='')
        for _ in res[1:]:
            print('%.5f'%_, ' ', end='')
        print()
        if type(res) == tuple:
            balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
            if res[0] not in stats:
                stats[res[0]] = 0
            stats[res[0]] += 1
print(balance)
print(stats)
if 'PROFIT' in stats:
    print("Success rate: %.2f%%" % (100.0*stats['PROFIT']/ sum(stats.values())))

2017-02-07 10:00:00     LOSS  -0.00154  0.00000  1.06597  1.07213  0.00226  
2017-02-10 16:00:00   PROFIT  0.00082  0.00000  1.06060  1.06748  0.00167  
2017-02-22 12:00:00   PROFIT  0.00082  0.00000  1.04924  1.05516  0.00164  
2017-06-15 19:00:00   PROFIT  0.00082  1.00000  1.11375  1.11851  0.00121  
0.00092
{'LOSS': 1, 'PROFIT': 3}
Success rate: 75.00%


<a id="eksperyment2"></a>
## Eksperyment 2) Spadające gwiazdy - EURUSD 1h

In [49]:
inst = INSTRUMENTS['EURUSD']

balance = 0.
stats = {}
for idx, row in inst.data['2017/01/01':'2017/07/01'].iterrows():
    if jestSpadajacaGwiazda(row) and zasada1120(inst,idx) and inst.rsi[idx] >= 70:
        res = sprawdz(inst, idx, 'sell') # XXX <- SELL
        if not res:
            print(idx, ' ', res)
            continue
        print(idx, ' ', '%6s'%res[0], ' ', end='')
        for _ in res[1:]:
            print('%.5f'%_, ' ', end='')
        print()
        if type(res) == tuple:
            balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
            if res[0] not in stats:
                stats[res[0]] = 0
            stats[res[0]] += 1
print(balance)  
print(stats)
if 'PROFIT' in stats:
    print("Success rate: %.2f%%" % (100.0*stats['PROFIT']/ sum(stats.values())))

2017-03-09 16:00:00   PROFIT  0.00082  0.00000  1.06171  1.05131  0.00354  
2017-04-12 23:00:00   PROFIT  0.00082  1.00000  1.06772  1.06132  0.00175  
2017-04-13 00:00:00     LOSS  -0.00096  0.00000  1.06751  1.06367  0.00108  
2017-06-14 17:00:00   PROFIT  0.00082  0.00000  1.12979  1.12139  0.00257  
0.0015
{'LOSS': 1, 'PROFIT': 3}
Success rate: 75.00%


<a id="eksperyment3"></a>
## Eksperyment 3) Analiza zbiorcza dla wszystkich walorów - Młotki

In [50]:
%%time

for name in INSTRUMENTS:
    print(name)
    inst = INSTRUMENTS[name]
    inst.results = pd.DataFrame( columns = range(7) )

    stats = {}
    balance = 0.
    for idx, row in inst.data['2017/01/01':].iterrows():  # XXX Month
        if jestMlotek(row) and zasada1120(inst,idx)  and inst.rsi[idx] <= 30: # XXX jestMlotek
            res = sprawdz(inst, idx) # XXX buy/sell
            if not res or type(res) == str:
                continue        
            inst.results = inst.results.append([ (idx,) + res ])
            if type(res) == tuple:
                balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
                if res[0] not in stats:
                    stats[res[0]] = 0
                stats[res[0]] += 1
    inst.results.columns = ['date', 'PL', 'gv', 'nswaps', 'sl', 'tp', 'size']
    inst.results.set_index('date', inplace=True)
    inst.balance = balance
    inst.stats = stats
    inst.success_rate = 0
    if 'PROFIT' in stats:
        inst.success_rate = stats['PROFIT']/ sum(stats.values())

EURSEK
AUDJPY
USDCHF
EURUSD
GBPCAD
EURAUD
AUDCAD
EURPLN
USDCAD
EURNOK
EURCHF
AUDCHF
AUDNZD
CADJPY
CPU times: user 15.7 s, sys: 16.7 ms, total: 15.7 s
Wall time: 15.7 s


In [51]:
for iname in sorted(INSTRUMENTS.keys()):
    inst = INSTRUMENTS[iname]
    if inst.results is not None:
        display(HTML('<h3>%s</h3>' % iname))
        display(HTML('<b>Balance: </b> %f &nbsp;&nbsp;&nbsp;&nbsp; <b>Trades:</b>%s' % (inst.balance, inst.stats)))
        display(HTML('<b>Success rate: </b> %.2f%%' % (100. * inst.success_rate)))
        print("One month details:")
        #display(inst.results['2017/06/01':])        
        display(inst.results[:])        

One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-05-04 00:00:00,LOSS,-0.00182,0.0,1.01766,1.02494,0.00160
2017-05-09 12:00:00,PROFIT,0.00068,0.0,1.00357,1.01121,0.00161
2017-06-09 16:00:00,LOSS,-0.00127,0.0,1.01175,1.01683,0.00124
2017-06-22 19:00:00,PROFIT,0.00068,1.0,0.99678,1.00454,0.00155


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-09 00:00:00,LOSS,-0.00129,0.0,0.76241,0.76757,0.00115
2017-03-27 17:00:00,PROFIT,0.00064,0.0,0.74725,0.75321,0.00159
2017-04-18 16:00:00,LOSS,-0.00159,0.0,0.75320,0.75956,0.00130
2017-04-18 20:00:00,LOSS,-0.00153,1.0,0.75169,0.75781,0.00126
2017-04-26 15:00:00,LOSS,-0.00161,0.0,0.74288,0.74932,0.00139
2017-04-26 17:00:00,LOSS,-0.00180,1.0,0.74154,0.74874,0.00136
2017-05-04 19:00:00,LOSS,-0.00124,0.0,0.73051,0.73547,0.00148
2017-05-31 18:00:00,LOSS,-0.00182,1.0,0.71859,0.72587,0.00165


One month details:


...

,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-27 10:00:00,PROFIT,0.00050,0.0,1.62355,1.63751,0.00331
2017-03-14 09:00:00,LOSS,-0.00383,0.0,1.63119,1.64651,0.00359
2017-03-14 10:00:00,PROFIT,0.00050,0.0,1.63122,1.64462,0.00338
2017-03-29 03:00:00,PROFIT,0.00050,0.0,1.65639,1.67675,0.00496
2017-04-07 13:00:00,PROFIT,0.00050,0.0,1.66194,1.67566,0.00293
2017-06-12 15:00:00,PROFIT,0.00050,0.0,1.70046,1.71282,0.00374
2017-06-13 08:00:00,PROFIT,0.00050,0.0,1.67937,1.69349,0.00305


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-16 02:00:00,PROFIT,0.00079,0.0,1.32812,1.33288,0.00136
2017-03-16 09:00:00,PROFIT,0.00079,0.0,1.32745,1.33757,0.00264
2017-03-21 15:00:00,PROFIT,0.00079,0.0,1.32620,1.33400,0.00239
2017-04-24 13:00:00,PROFIT,0.00079,0.0,1.34089,1.34789,0.00221
2017-06-22 17:00:00,PROFIT,0.00079,0.0,1.32063,1.32943,0.00295


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-16 02:00:00,PROFIT,0.00080,0.0,0.99825,1.00317,0.00167
2017-03-16 08:00:00,LOSS,-0.00120,0.0,0.99801,1.00281,0.00142
2017-03-27 10:00:00,LOSS,-0.00197,0.0,0.98341,0.99129,0.00167
2017-03-27 11:00:00,LOSS,-0.00140,0.0,0.98409,0.98969,0.00122
2017-03-27 16:00:00,PROFIT,0.00080,0.0,0.98116,0.99004,0.00211
2017-05-16 14:00:00,LOSS,-0.00211,0.0,0.98899,0.99743,0.00196
2017-05-17 17:00:00,PROFIT,0.00080,0.0,0.97726,0.98418,0.00260
2017-05-31 14:00:00,PROFIT,0.00080,0.0,0.96989,0.97533,0.00123
2017-06-05 00:00:00,PROFIT,0.00080,0.0,0.96081,0.97009,0.00241


In [57]:
INSTRUMENTS['USDCHF'].results['gv'].sum()

-0.0026799999999999971

<a id="eksperyment3summary"></a>
### Podsumowanie zbiorcze - Młotki

In [58]:
posivite_instruments = 0
for iname in sorted(INSTRUMENTS.keys()):
    print('%11s   %g' % (iname, INSTRUMENTS[iname].balance))
    if INSTRUMENTS[iname].balance > 0:
        posivite_instruments += 1
print()
print("%d/%d (%.2f%%)" % (posivite_instruments, len(INSTRUMENTS),  100. * posivite_instruments / len(INSTRUMENTS)))
print()

total_nloss = 0
total_nprofit = 0
for iname in sorted(INSTRUMENTS.keys()):
    total_nloss += INSTRUMENTS[iname].stats.get('LOSS', 0)
    total_nprofit += INSTRUMENTS[iname].stats.get('PROFIT', 0)
    #print(iname, str(INSTRUMENTS[iname].stats))
    #if INSTRUMENTS[iname].balance > 0:
    #    posivite_instruments += 1
print("%d %d" % (total_nprofit, total_nprofit + total_nloss))

     AUDCAD   -0.00173
     AUDCHF   -0.01024
     AUDJPY   -0.056
     AUDNZD   0.00118
     CADJPY   -0.155
     EURAUD   -0.0021
     EURCHF   0
     EURNOK   -0.06388
     EURPLN   -0.04033
     EURSEK   -0.0031
     EURUSD   0.00092
     GBPCAD   -0.00083
     USDCAD   0.00395
     USDCHF   -0.00268

3/14 (21.43%)

48 67


<a id='eksperyment4'></a>
## Eksperyment 4) Analiza zbiorcza dla wszystkich walorów - Spadające gwiazdy

In [40]:
%%time

for iname in sorted(INSTRUMENTS.keys()):
    print(iname)
    inst = INSTRUMENTS[iname]
    inst.results = pd.DataFrame( columns = range(7) )

    stats = {}
    balance = 0.
    for idx, row in inst.data['2017/01/01':].iterrows():  # XXX Month
        if jestSpadajacaGwiazda(row) and zasada1120(inst,idx) and inst.rsi[idx] >= 70: # XXX jestMlotek
            res = sprawdz(inst, idx, 'sell') # XXX buy/sell
            if not res or type(res) == str:
                continue        
            inst.results = inst.results.append([ (idx,) + res ])
            if type(res) == tuple:
                balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
                if res[0] not in stats:
                    stats[res[0]] = 0
                stats[res[0]] += 1
    inst.results.columns = ['date', 'PL', 'gv', 'nswaps', 'sl', 'tp', 'size']
    inst.results.set_index('date', inplace=True)
    inst.balance = balance
    inst.stats = stats
    inst.success_rate = 0
    if 'PROFIT' in stats:
        inst.success_rate = stats['PROFIT']/ sum(stats.values())

AUDCAD
AUDCHF
AUDJPY
AUDNZD
CADJPY
EURAUD
EURCHF
EURNOK
EURPLN
EURSEK
EURUSD
GBPCAD
USDCAD
USDCHF
CPU times: user 15.3 s, sys: 16.6 ms, total: 15.3 s
Wall time: 15.3 s


In [41]:
for iname in sorted(INSTRUMENTS.keys()):
    inst = INSTRUMENTS[iname]
    if inst.results is not None:
        display(HTML('<h3>%s</h3>' % iname))
        display(HTML('<b>Balance: </b> %f &nbsp;&nbsp;&nbsp;&nbsp; <b>Trades:</b>%s' % (inst.balance, inst.stats)))
        display(HTML('<b>Success rate: </b> %.2f%%' % (100. * inst.success_rate)))
        print("One month details:")
        #display(inst.results['2017/06/01':])        
        display(inst.results[:])        

One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-01 04:00:00,PROFIT,0.00102,0.0,1.02324,1.01000,0.00306
2017-05-01 14:00:00,LOSS,-0.00193,0.0,1.02839,1.02067,0.00161
2017-05-01 15:00:00,LOSS,-0.00185,0.0,1.02912,1.02172,0.00194
2017-05-01 17:00:00,PROFIT,0.00103,0.0,1.03079,1.02131,0.00206
2017-06-05 17:00:00,PROFIT,0.00108,0.0,1.01055,1.00411,0.00141
2017-06-23 16:00:00,PROFIT,0.00181,0.0,1.00698,0.99814,0.00198


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-07 07:00:00,PROFIT,0.00106,0.0,0.77181,0.76489,0.00098
2017-03-28 21:00:00,LOSS,-0.00221,1.0,0.75930,0.75046,0.00197


One month details:


...

,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-04-18 20:00:00,PROFIT,0.00097,0.0,1.72830,1.68010,0.01846
2017-04-19 15:00:00,PROFIT,0.00131,0.0,1.72853,1.70773,0.00439
2017-04-19 16:00:00,PROFIT,0.00136,0.0,1.72625,1.71033,0.00323
2017-04-25 12:00:00,PROFIT,0.00104,0.0,1.73991,1.72563,0.00262


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-02-28 20:00:00,LOSS,-0.00208,0.0,1.32892,1.32060,0.00157
2017-04-19 15:00:00,LOSS,-0.00145,0.0,1.34456,1.33876,0.00151


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-05-10 18:00:00,PROFIT,0.00134,1.0,1.00972,1.005,0.0013


<a id="eksperyment4summary"></a>
### Podsumowanie zbiorcze - spadające gwiazdy

In [42]:
posivite_instruments = 0
for iname in sorted(INSTRUMENTS.keys()):
    print('%11s   %g' % (iname, INSTRUMENTS[iname].balance))
    if INSTRUMENTS[iname].balance > 0:
        posivite_instruments += 1
print()
print("%d/%d (%.2f%%)" % (posivite_instruments, len(INSTRUMENTS),  100. * posivite_instruments / len(INSTRUMENTS)))
print()

total_nloss = 0
total_nprofit = 0
for iname in sorted(INSTRUMENTS.keys()):
    total_nloss += INSTRUMENTS[iname].stats.get('LOSS', 0)
    total_nprofit += INSTRUMENTS[iname].stats.get('PROFIT', 0)
    #print(iname, str(INSTRUMENTS[iname].stats))
    #if INSTRUMENTS[iname].balance > 0:
    #    posivite_instruments += 1
print("%d %d" % (total_nprofit, total_nprofit + total_nloss))

     AUDCAD   0.00116
     AUDCHF   -0.00115
     AUDJPY   0.1378
     AUDNZD   0.00127
     CADJPY   0.0718
     EURAUD   0.00211
     EURCHF   0.00459
     EURNOK   0.0937
     EURPLN   0.00512
     EURSEK   0.00906
     EURUSD   0.00309
     GBPCAD   0.00468
     USDCAD   -0.00353
     USDCHF   0.00134

12/14 (85.71%)

41 51


In [79]:
INSTRUMENTS['AUDCAD'].spec

Description      AUSTRALIAN DOLLAR VS CANADIAN DOLLAR
Digits                                              5
Spread                                             32
Swap Long                                        -0.9
Swap Short                                      -8.19
Swap Type                                      points
Stop Level                                        160
Lot Size                                       100000
Lot Min                                          0.01
Lot Step                                         0.01
Tick Value                                    2.83951
Tick Size                                       1e-05
Trade Allowed                                     yes
Pr. Mode                                        Forex
Pip                                             1e-05
Name: AUDCAD, dtype: object